In [1]:
import pandas as pd
import numpy as np

import requests

In [2]:
df = pd.read_csv('ProcessedDataset.csv')
df

,Unnamed: 0,Наименование,Скидка,Цена,Цена со скидкой,Бренд,feedbacks,rating,Ссылка,high_rating,memory_size
0,0,"9 5G 4 6.6"" 120 герц",36,21990,14073,Realme,1116,5,https://www.wildberries.ru/catalog/121028588/d...,1,128Gb
1,2,Reno 7 8,43,27990,15954,OPPO,357,5,https://www.wildberries.ru/catalog/116872732/d...,1,128Gb
2,3,Reno 7 8,43,27990,15954,OPPO,288,5,https://www.wildberries.ru/catalog/116872733/d...,1,128Gb
3,4,SMART 6 PLUS 2,40,9990,5994,Infinix,1813,5,https://www.wildberries.ru/catalog/114477948/d...,1,64Gb
4,5,NOTE 12 2023 8,35,22990,14943,Infinix,118,5,https://www.wildberries.ru/catalog/128728547/d...,1,256Gb
...,...,...,...,...,...,...,...,...,...,...,...
1114,2740,M5 4 6.58 FHD MTK G99,18,16990,13931,POCO,2,5,https://www.wildberries.ru/catalog/106176120/d...,1,128Gb
1115,2761,Redmi 10C,31,18990,13103,Xiaomi,2,5,https://www.wildberries.ru/catalog/90422386/de...,1,64Gb
1116,2763,Galaxy S5 SM-G900F,0,10990,10990,Samsung,1,1,https://www.wildberries.ru/catalog/96896454/de...,0,16Gb
1117,2771,"C25s 6,5"" 4 64гб HD 6000 мАч NFC",0,12990,12990,Realme,1,5,https://www.wildberries.ru/catalog/46016394/de...,1,NaN


In [7]:
def get_id_from_link(link_name):
    id_value = link_name.split('catalog/', 1)[1]
    id_value = id_value.split('/detail', 1)[0]
    return id_value

In [40]:
ids = df['Ссылка'].apply(get_id_from_link)
ids

0       121028588
1       116872732
2       116872733
3       114477948
4       128728547
          ...    
1114    106176120
1115     90422386
1116     96896454
1117     46016394
1118     94669604
Name: Ссылка, Length: 1119, dtype: object

In [24]:
def get_card_json_url(id_value):
    url = ''
    if len(id_value) == 7:
        url = f'https://basket-01.wb.ru/vol{id_value[:2]}/part{id_value[:4]}/{id_value}/info/ru/card.json'
    if len(id_value) == 8:
        if 100 <= int(id_value[:3]) <= 144:
            url = f'https://basket-01.wb.ru/vol{id_value[:3]}/part{id_value[:5]}/{id_value}/info/ru/card.json'
        if 144 < int(id_value[:3]) <= 280:
            url = f'https://basket-02.wb.ru/vol{id_value[:3]}/part{id_value[:5]}/{id_value}/info/ru/card.json'
        if 280 < int(id_value[:3]) <= 450:
            url = f'https://basket-03.wb.ru/vol{id_value[:3]}/part{id_value[:5]}/{id_value}/info/ru/card.json'            
        if 450 < int(id_value[:3]) <= 736:
            url = f'https://basket-04.wb.ru/vol{id_value[:3]}/part{id_value[:5]}/{id_value}/info/ru/card.json'            
        if 736 < int(id_value[:3]) <= 999:
            url = f'https://basket-05.wb.ru/vol{id_value[:3]}/part{id_value[:5]}/{id_value}/info/ru/card.json'

    if len(id_value) == 9:
        if 1000 <= int(id_value[:4]) <= 1020:
            url = f'https://basket-05.wb.ru/vol{id_value[:4]}/part{id_value[:6]}/{id_value}/info/ru/card.json'           
        if 1020 < int(id_value[:4]) <= 1060:
            url = f'https://basket-06.wb.ru/vol{id_value[:4]}/part{id_value[:6]}/{id_value}/info/ru/card.json'            
        if 1060 < int(id_value[:4]) <= 1179:
            url = f'https://basket-08.wb.ru/vol{id_value[:4]}/part{id_value[:6]}/{id_value}/info/ru/card.json'            
        if 1180 < int(id_value[:4]):
            url = f'https://basket-09.wb.ru/vol{id_value[:4]}/part{id_value[:6]}/{id_value}/info/ru/card.json'            
    return url

In [25]:
def get_sales_count_url(id_value):
    return 'https://product-order-qnt.wildberries.ru/by-nm/?nm='+id_value

In [81]:
def get_data_from_card_json(json_data):
    data_dict = {}
    data_dict['id'] = json_data['nm_id']
    
    curr_dict = find_dict_in_list_with_key(json_data['grouped_options'], 'Дополнительная информация')
    data_dict['country'] = find_value_in_list_with_key(curr_dict['options'], 'Страна производства')
    data_dict['weight'] = find_value_in_list_with_key(curr_dict['options'], 'Вес товара без упаковки (г)')        
    data_dict['width'] = find_value_in_list_with_key(curr_dict['options'], 'Ширина предмета')    
    data_dict['height'] = find_value_in_list_with_key(curr_dict['options'], 'Высота предмета')                
    data_dict['thickness'] = find_value_in_list_with_key(curr_dict['options'], 'Толщина предмета')
    
    curr_dict = find_dict_in_list_with_key(json_data['grouped_options'], 'Экран')    
    data_dict['screen_resol'] = find_value_in_list_with_key(curr_dict['options'], 'Разрешение экрана')
    data_dict['screen_size'] = find_value_in_list_with_key(curr_dict['options'], 'Диагональ экрана')        
    data_dict['screen_type'] = find_value_in_list_with_key(curr_dict['options'], 'Тип дисплея/экрана')    
    return data_dict

In [82]:
def find_dict_in_list_with_key(input_list, key):
    for curr_dict in input_list:
        if curr_dict['group_name'] == key:
            return curr_dict
    return None

In [83]:
def find_value_in_list_with_key(input_list, key):
    for curr_dict in input_list:
        if curr_dict['name'] == key:
            return curr_dict['value']
    return None

In [84]:
data_dict = {}
for id_num in range(3):
    r = requests.get(get_card_json_url(ids[id_num]))
    data = r.json()
    curr_data_dict = get_data_from_card_json(data)
    r = requests.get(get_sales_count_url(ids[id_num]))
    data = r.json()
    curr_data_dict['sales_count'] = data[0]['qnt']
    data_dict[id_num] = curr_data_dict

extended_df = pd.DataFrame.from_dict(data_dict, orient='index')

In [85]:
extended_df

,id,country,weight,width,height,thickness,screen_resol,screen_size,screen_type,sales_count
0,121028588,Китай,191 г,75.6 см,164.3 см,8.5 см,2412x1080,6.6,Super Clear LCD,8600
1,116872732,Китай,175 г,7.32 см,None,0.78 см,2400x1080,6.40,Безрамочный дисплей,4100
2,116872733,Китай,175 г,7.32 см,None,0.78 см,2400 x 1080,6.40,Безрамочный дисплей,4100


In [28]:
get_card_json_url(ids[0])

'https://basket-09.wb.ru/vol1210/part121028/121028588/info/ru/card.json'

**Ниже все параметры, которые есть в card. Я вытащил только часть из них, возможно потребуется что-то еще**

In [44]:
from pprint import pprint

r_1 = requests.get(get_card_json_url(ids[0]))
data_1 = r_1.json()
pprint(data_1)

{'certificate': {},
 'colors': [121028588, 121028587],
 'contents': 'чехол; Смартфон, кабель USB type-C, зарядное устройство, '
             'защитное стекло, инструкция по эксплуатации',
 'data': {'chrt_ids': [213916452],
          'skus': ['6941399096967'],
          'subject_id': 515,
          'subject_root_id': 6258,
          'tech_size': '0'},
 'description': 'огда свет попадает на realme 9 5G, корпус смартфона сияет '
                'различными оттенками цветов, создавая постоянный '
                'волнообразный голографический эффект. При весе 191 г. и '
                'толщине 8,5 мм realme 9 5G легко помещается в вашей руке. Он '
                'будет сопровождать вас, куда бы вы ни отправились.Экран '
                'realme 9 5G с диагональю 6,6 дюйма имеет соотношение экрана к '
                'корпусу 90,8%, для захватывающего просмотра контента. Частота '
                'обновления 120 Гц и частота дискретизации 240 Гц делают '
                'каждый свайп и про

In [74]:
r_1 = requests.get(get_card_json_url(ids[1]))
data_1 = r_1.json()
pprint(data_1)

{'certificate': {},
 'colors': [116872732],
 'contents': 'смартфон; гарантийный талон; адаптер питания; Кабель USB Type-C; '
             'Скрепка для извлечения SIM-карты',
 'data': {'chrt_ids': [208698743],
          'skus': ['6932169307437'],
          'subject_id': 515,
          'subject_root_id': 6258,
          'tech_size': '0'},
 'description': 'Мое безграничное творчество! Великолепный OPPO Reno7 оснащен '
                'сверхчувствительной селфи-камерой с датчиком изображения '
                'IMX709 и портретным режимом с эффектом Bokeh Flare. Стильный '
                'внешний вид благодаря новому дизайну корпуса с имитацией кожи '
                'и ультратонкому ретродизайну. Долговечность и качество OPPO '
                'гарантируют безупречную работу и удобство пользования.',
 'full_colors': [{'nm_id': 116872732}],
 'grouped_options': [{'group_name': 'Дополнительная информация',
                      'options': [{'name': 'Высота упаковки',
                        

In [49]:
data_1['grouped_options'][0]

{'group_name': 'Дополнительная информация',
 'options': [{'name': 'Вес товара с упаковкой (г)', 'value': '450 г'},
  {'name': 'Толщина предмета', 'value': '8.5 см'},
  {'name': 'Вес товара без упаковки (г)', 'value': '191 г'},
  {'name': 'Высота упаковки', 'value': '17.7 см'},
  {'name': 'Длина упаковки', 'value': '6.9 см'},
  {'name': 'Ширина упаковки', 'value': '10 см'},
  {'name': 'Ширина предмета', 'value': '75.6 см'},
  {'name': 'Доп. опции смартфона', 'value': 'Быстрая зарядка 18Вт'},
  {'name': 'Страна производства', 'value': 'Китай'},
  {'name': 'Высота предмета', 'value': '164.3 см'}]}

In [63]:
find_dict_in_list_with_key(data_1['grouped_options'][0]['options'], 'Толщина предмета')

'8.5 см'

In [50]:
data_1['grouped_options'][1]

{'group_name': 'Экран',
 'options': [{'name': 'Защитное покрытие экрана',
   'value': 'Corning Gorilla Glass Victus; Олеофобное'},
  {'name': 'Разрешение экрана', 'value': '2412x1080'},
  {'name': 'Диагональ экрана', 'value': '6.6'},
  {'name': 'Тип дисплея/экрана', 'value': 'Super Clear LCD'}]}

In [51]:
data_1['grouped_options'][2]

{'group_name': 'Память',
 'options': [{'name': 'Максимальный объем карты памяти', 'value': '1 ТБ'},
  {'name': 'Объем встроенной памяти (Гб)', 'value': '128 GB'},
  {'name': 'Объем оперативной памяти (Мб)', 'value': '4000'},
  {'name': 'Тип карты памяти', 'value': 'MicroSD (до 1ТБ)'},
  {'name': 'Объем оперативной памяти (Гб)', 'value': '4 GB'}]}

In [52]:
data_1['grouped_options'][3]

{'group_name': 'Процессор',
 'options': [{'name': 'Количество ядер процессора', 'value': '8'},
  {'name': 'Тактовая частота процессора', 'value': '2200 МГц'},
  {'name': 'Процессор', 'value': 'Snapdragon 695'}]}

In [53]:
data_1['grouped_options'][4]

{'group_name': 'Мультимедийные возможности',
 'options': [{'name': 'Встроенная вспышка', 'value': 'LED-вспышка'},
  {'name': 'Количество мп основной камеры', 'value': '50 Мп'},
  {'name': 'Основная камера (млн. пикс.)', 'value': '50 Мп'},
  {'name': 'Особенности объектива', 'value': 'Портретный модуль'},
  {'name': 'Фронтальная камера (млн. пикс.)', 'value': '16 Мп'},
  {'name': 'Вторая основная камера (млн. пикс.)', 'value': '2 Мп'},
  {'name': 'Доп. опции камеры',
   'value': 'Распознавание лиц; Портретный режим, Ночной режим'},
  {'name': 'Третья основная камера', 'value': '2 Мп'}]}

In [54]:
data_1['grouped_options'][5]

{'group_name': 'Связь',
 'options': [{'name': 'Количество SIM карт', 'value': '2 nano SIM'},
  {'name': 'Беспроводные интерфейсы', 'value': 'Bluetooth; GPS; NFC; Wi-Fi'},
  {'name': 'Стандарт связи', 'value': 'GSM 1801; 2G, 3G, 5G, LTE'},
  {'name': 'Спутниковая навигация',
   'value': 'BeiDou , Galileo , A-GPS , GPS , ГЛОНАСС'}]}

In [55]:
data_1['grouped_options'][6]

{'group_name': 'Питание',
 'options': [{'name': 'Время работы при прослушивании музыки',
   'value': '122 час'},
  {'name': 'Емкость аккумулятора', 'value': '5000 m*Ah'},
  {'name': 'Время работы при воспроизведении видео', 'value': '22 час'}]}

In [56]:
data_1['grouped_options'][7]

{'group_name': 'Общие характеристики',
 'options': [{'name': 'Операционная система', 'value': 'Android 12'},
  {'name': 'Модель', 'value': 'realme 9 5G'},
  {'name': 'Тип SIM карты', 'value': 'Nano-SIM'},
  {'name': 'Гарантийный срок', 'value': '1 (один) год'},
  {'name': 'Версия операционной системы', 'value': 'Android 12'}]}

In [57]:
data_1['grouped_options'][8]

{'group_name': 'Другие функции',
 'options': [{'name': 'Поддерживаемый формат',
   'value': 'AAC/APE/FLAC/AMR/MID/MP3/OGG/WAV/WMA/MKA'},
  {'name': 'Хрупкость', 'value': 'хрупкое'}]}

In [58]:
data_1['grouped_options'][9]

{'group_name': 'Интерфейсы и разъемы',
 'options': [{'name': 'Вид разъема', 'value': 'typ-c'}]}

In [59]:
data_1['options']

[{'name': 'Вес товара с упаковкой (г)', 'value': '450 г'},
 {'name': 'Защитное покрытие экрана',
  'value': 'Corning Gorilla Glass Victus; Олеофобное'},
 {'name': 'Максимальный объем карты памяти', 'value': '1 ТБ'},
 {'name': 'Толщина предмета', 'value': '8.5 см'},
 {'name': 'Объем встроенной памяти (Гб)', 'value': '128 GB'},
 {'name': 'Разрешение экрана', 'value': '2412x1080'},
 {'name': 'Вес товара без упаковки (г)', 'value': '191 г'},
 {'name': 'Высота упаковки', 'value': '17.7 см'},
 {'name': 'Количество ядер процессора', 'value': '8'},
 {'name': 'Встроенная вспышка', 'value': 'LED-вспышка'},
 {'name': 'Объем оперативной памяти (Мб)', 'value': '4000'},
 {'name': 'Длина упаковки', 'value': '6.9 см'},
 {'name': 'Количество SIM карт', 'value': '2 nano SIM'},
 {'name': 'Беспроводные интерфейсы', 'value': 'Bluetooth; GPS; NFC; Wi-Fi'},
 {'name': 'Диагональ экрана', 'value': '6.6'},
 {'name': 'Ширина упаковки', 'value': '10 см'},
 {'name': 'Время работы при прослушивании музыки', 'value'

In [42]:
data_1['nm_id']

121028588

In [37]:
r_2 = requests.get(get_sales_count_url(ids[0]))
data_2 = r_2.json()
print(data_2)

[{'nmId': 121028588, 'qnt': 8600}]
